<a href="https://colab.research.google.com/github/sai-phyo-hein/Hybrid_Transformer_for_Portfolio_Allocation/blob/main/Stock_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import duckdb as ddb
import pandas as pd

In [ ]:
con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_commodity_dataset.db')
print(con.sql('select * from duckdb_tables').df()['table_name'])
con.close()

0            ACC
1     ADANIENSOL
2     ADANIGREEN
3     ADANIPOWER
4      AMBUJACEM
5      APLAPOLLO
6           BPCL
7      COALINDIA
8      DALBHARAT
9      DEEPAKNTR
10        GRASIM
11      HINDALCO
12     HINDPETRO
13           IOC
14    JINDALSTEL
15      JSWSTEEL
16          NTPC
17          ONGC
18    PIDILITIND
19         PIIND
20      RELIANCE
21          SAIL
22      SHREECEM
23           SRF
24      TATACHEM
25     TATAPOWER
26     TATASTEEL
27    ULTRACEMCO
28           UPL
29          VEDL
Name: table_name, dtype: object


In [ ]:
con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_50_dataset.db')
print(con.sql('select * from duckdb_tables').df()['table_name'])
con.close()

0       ADANIENT
1     ADANIPORTS
2     APOLLOHOSP
3     ASIANPAINT
4       AXISBANK
5     BAJAJFINSV
6     BAJAJ_AUTO
7     BAJFINANCE
8     BHARTIARTL
9           BPCL
10     BRITANNIA
11         CIPLA
12     COALINDIA
13      DIVISLAB
14       DRREDDY
15     EICHERMOT
16        GRASIM
17       HCLTECH
18      HDFCBANK
19      HDFCLIFE
20    HEROMOTOCO
21      HINDALCO
22    HINDUNILVR
23     ICICIBANK
24    INDUSINDBK
25          INFY
26           ITC
27      JSWSTEEL
28     KOTAKBANK
29            LT
30          LTIM
31        MARUTI
32     NESTLEIND
33          NTPC
34          ONGC
35     POWERGRID
36      RELIANCE
37       SBILIFE
38          SBIN
39    SHRIRAMFIN
40     SUNPHARMA
41    TATACONSUM
42    TATAMOTORS
43     TATASTEEL
44           TCS
45         TECHM
46         TITAN
47    ULTRACEMCO
48         WIPRO
Name: table_name, dtype: object


In [ ]:
con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_50_dataset.db')
print(con.sql('select * from TITAN limit 1').df().columns)
con.close()

Index(['_id', 'CH_SYMBOL', 'CH_SERIES', 'CH_MARKET_TYPE',
       'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE',
       'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE',
       'CH_TOT_TRADED_QTY', 'CH_TOT_TRADED_VAL', 'CH_52WEEK_HIGH_PRICE',
       'CH_52WEEK_LOW_PRICE', 'CH_TOTAL_TRADES', 'CH_ISIN', 'CH_TIMESTAMP',
       'TIMESTAMP', 'createdAt', 'updatedAt', '__v', 'SLBMH_TOT_VAL', 'VWAP',
       'mTIMESTAMP', 'CA'],
      dtype='object')


### Equity Selection

In [ ]:
tickers = []
date_min = []
date_max = []
mean_volume = []
data_points = []
eq_selection_df = pd.DataFrame()

con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_50_dataset.db')
for ticker in con.sql('select * from duckdb_tables').df()['table_name']:
    tickers.append(ticker)
    temp_df = con.sql(
        f"""
        select min(TIMESTAMP) as min_, max(TIMESTAMP) as max_, count(_id) as shape_, mean(cast(CH_TOTAL_TRADES as float)) as mean_volume from {ticker}
        """
    ).df()
    date_min.append(temp_df.min_.values[0])
    date_max.append(temp_df.max_.values[0])
    data_points.append(temp_df.shape_.values[0])
    mean_volume.append(temp_df.mean_volume.values[0])

con.close()
eq_selection_df['tickers'] = tickers
eq_selection_df['date_min'] = date_min
eq_selection_df['date_max'] = date_max
eq_selection_df['data_points'] = data_points
eq_selection_df['mean_volume'] = mean_volume

In [ ]:
sorted(eq_selection_df.date_min.unique().tolist())

['2013-12-31T18:30:00.000Z',
 '2016-07-20T18:30:00.000Z',
 '2017-10-02T18:30:00.000Z',
 '2017-11-16T18:30:00.000Z']

In [ ]:
sorted(eq_selection_df.date_max.unique().tolist())

['2024-03-27T18:30:00.000Z']

In [ ]:
eq_selection_df = eq_selection_df[eq_selection_df.date_min == '2013-12-31T18:30:00.000Z']
eq_selection_df.tickers.nunique()

46

In [ ]:
eq_selection_df.data_points.describe()

count      46.000000
mean     2614.826087
std         2.293027
min      2610.000000
25%      2613.000000
50%      2615.000000
75%      2616.000000
max      2620.000000
Name: data_points, dtype: float64

In [ ]:
eq_selection_df.mean_volume.describe()

count        46.000000
mean      79879.761979
std       44713.290164
min       16736.534635
25%       44662.167126
50%       70584.422737
75%       95279.596521
max      187491.141491
Name: mean_volume, dtype: float64

In [ ]:
eq_selection_df[eq_selection_df.mean_volume > 90e3].shape

(14, 5)

### Commodity Selection

In [ ]:
tickers = []
date_min = []
date_max = []
mean_volume = []
data_points = []
commo_selection_df = pd.DataFrame()

con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_commodity_dataset.db')
for ticker in con.sql('select * from duckdb_tables').df()['table_name']:
    tickers.append(ticker)
    temp_df = con.sql(
        f"""
        select min(TIMESTAMP) as min_, max(TIMESTAMP) as max_, count(_id) as shape_, mean(cast(CH_TOTAL_TRADES as float)) as mean_volume from {ticker}
        """
    ).df()
    date_min.append(temp_df.min_.values[0])
    date_max.append(temp_df.max_.values[0])
    data_points.append(temp_df.shape_.values[0])
    mean_volume.append(temp_df.mean_volume.values[0])

con.close()
commo_selection_df['tickers'] = tickers
commo_selection_df['date_min'] = date_min
commo_selection_df['date_max'] = date_max
commo_selection_df['data_points'] = data_points
commo_selection_df['mean_volume'] = mean_volume

In [ ]:
commo_selection_df.head()

,tickers,date_min,date_max,data_points,mean_volume
0,ACC,2013-12-31T18:30:00.000Z,2024-03-27T18:30:00.000Z,2611,26036.130601
1,ADANIENSOL,2015-08-13T18:30:00.000Z,2024-03-27T18:30:00.000Z,2016,20422.510417
2,ADANIGREEN,2018-07-01T18:30:00.000Z,2024-03-27T18:30:00.000Z,1410,40915.642553
3,ADANIPOWER,2013-12-31T18:30:00.000Z,2024-03-27T18:30:00.000Z,2401,41078.954186
4,AMBUJACEM,2013-12-31T18:30:00.000Z,2024-03-27T18:30:00.000Z,2617,45033.693924


In [ ]:
sorted(commo_selection_df.date_min.unique().tolist())

['2013-12-31T18:30:00.000Z',
 '2014-01-01T18:30:00.000Z',
 '2015-08-13T18:30:00.000Z',
 '2018-07-01T18:30:00.000Z',
 '2019-02-04T18:30:00.000Z']

In [ ]:
sorted(commo_selection_df.date_max.unique().tolist())

['2024-03-27T18:30:00.000Z']

In [ ]:
commo_selection_df = commo_selection_df[commo_selection_df.date_min == '2013-12-31T18:30:00.000Z']
commo_selection_df.tickers.nunique()

26

In [ ]:
commo_selection_df.data_points.describe()

count      26.000000
mean     2607.000000
std        42.119829
min      2401.000000
25%      2613.000000
50%      2615.500000
75%      2617.000000
max      2622.000000
Name: data_points, dtype: float64

In [ ]:
commo_selection_df.mean_volume.describe()

count        26.000000
mean      54543.566637
std       37542.069383
min        9631.944593
25%       28718.813463
50%       51305.962108
75%       65800.996802
max      184568.369549
Name: mean_volume, dtype: float64

In [ ]:
commo_selection_df[commo_selection_df.mean_volume > 65e3].shape

(7, 5)